# CPS Report

The following notebook requires Ubuntu 22.04 LTS (Jammy Jellyfish) to be executed

### Preliminary installations with ROS 2 Humble/Ubuntu 22.04 

https://docs.ros.org/en/humble/Tutorials.html

https://docs.ros.org/en/humble/Installation/Ubuntu-Install-Debians.html

In [ ]:
!sudo apt install software-properties-common
!sudo add-apt-repository universe

In [ ]:
!sudo curl -sSL https://raw.githubusercontent.com/ros/rosdistro/master/ros.key -o /usr/share/keyrings/ros-archive-keyring.gpg
!echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/ros-archive-keyring.gpg] http://packages.ros.org/ros2/ubuntu $(. /etc/os-release && echo $UBUNTU_CODENAME) main" | sudo tee /etc/apt/sources.list.d/ros2.list > /dev/null

In [ ]:
# ROS installation
!sudo apt update
!sudo apt upgrade
!sudo apt install ros-humble-desktop

!sudo apt install ros-dev-tools

# Gazebo installation
!sudo apt install libgazebo-dev
!sudo apt install ros-humble-gazebo-ros-pkgs

In [ ]:
# ALWAYS REMEMBER TO SOURCE THIS FILE (or add it in the .bashrc)
!source /opt/ros/humble/setup.bash

!echo "source /opt/ros/humble/setup.bash" >> ~/.bashrc

https://docs.px4.io/main/en/dev_setup/dev_env_linux_ubuntu.html#ros-2

In [ ]:
# download PX4
!git clone https://github.com/PX4/PX4-Autopilot.git --recursive

# install
!bash ./PX4-Autopilot/Tools/setup/ubuntu.sh -no-sim-tools --no-nuttx

# then restart computer

# additional dependencies
!sudo apt-get install protobuf-compiler libeigen3-dev libopencv-dev -y

### .SDF to .URDF conversion of the iris model

Since Rviz requires the drone's model to be in .urdf format, it is mandatory to convert the .sdf file of it to be readable.

### Generic PX4 Gazebo simulation with IRIS drone

In [ ]:
!cd PX4-Autopilot
!make px4_sitl gazebo-classic

### PX4 Gazebo simulation with IRIS depth camera .urdf file

In [ ]:
!cd PX4_Autopilot
!make px4_sitl gazebo_iris_depth_camera

### ROS2 PX4 Offboard script

https://github.com/ARK-Electronics/ROS2_PX4_Offboard_Example

In [ ]:
# Micro XRCE installation
!pip3 install --user -U empy pyros-genmsg setuptools
!pip3 install kconfiglib
!pip install --user jsonschema
!pip install --user jinja2

!git clone https://github.com/eProsima/Micro-XRCE-DDS-Agent.git
!cd Micro-XRCE-DDS-Agent
!mkdir build
!cd build
!cmake ..
!make
!sudo make install
!sudo ldconfig /usr/local/lib/

In [ ]:
# Inside the /ROS2_ws/src directory
!git clone https://github.com/PX4/px4_msgs.git
!git clone https://github.com/ARK-Electronics/ROS2_PX4_Offboard_Example.git

!cd ..
!pip install --user -U empy==3.3.4 pyros-genmsg setuptools
!colcon build

To use a different IRIS model file, as our .urdf one, we need to change some lines in ROS2_ws/src/ROS2_PX4_Offboard_Example/px4_offboard/px4_offboard/processes.py:

 Run the PX4 SITL simulation 

  "cd ~/PX4-Autopilot && make px4_sitl gz_x500"

to:

 Run the PX4 SITL simulation 
 
  "cd ~/PX4-Autopilot && make px4_sitl gazebo-classic_iris_depth_camera__warehouse" for the warehouse world
  "cd ~/PX4-Autopilot && make px4_sitl gazebo-classic_iris_depth_camera__our_empty" for our simple world

In [ ]:
# Source the code every time -> from /ROS2_ws
!source . install/setup.bash

The simulation will be launched with all the required processes, to add new topics or nodes it will be easier to add the corresponding commands directly in the .launch file.

In [ ]:
# To run the simulation
!ros2 launch px4_offboard offboard_velocity_control.launch.py

At this point we are able to launch Gazebo and Rviz2 with a depth camera equipped drone, publishing the image of it and the associated point cloud that will be used to map the environment.

In our project we faced a problem with the ROS topic correct publishing, where the data related to the sensor wasn't correctly linked to the map frame, leading to an incompatibility between the sensor itself and the octomap library. To solve this issue we needed to manually publish the topics and link it to the map frame, with the correct transformation.

### Base_Link and Map publishing

In [ ]:
# In /ROS2_ws, when the simulation in gazebo and rviz is running
!source . install/setup.bash
!ros2 run map_base_link_publ map_base_link_publ
!ros2 run map_frame_publisher map_frame_publisher

### PointCloud publishing wrt Map

In addition we need to execute one last script in order to transform the pointcloud with respect to the map frame and use it for the mapping process.

In [ ]:
# In /ROS2_ws
!source . install/setup.bash
!ros2 run pointcloud_transformer pointcloud_transformer

For ease, all these three executions will be added to the launch file.

### OctoMap Mapping

Version for ROS 2 Humble

In [ ]:
!sudo apt-get install ros-humble-octomap-mapping

!sudo nautilus # to modify the launch file as needed
# The data will then be located at /opt/ros/humble/share/octomap_server/launch/ --> insert the topic /pointcloud_wrt_map and the frame_id as map

!ros2 launch octomap_server octomap_mapping.launch.xml

### Multiagent simulation with PX4 in Gazebo

https://docs.px4.io/v1.12/en/simulation/multi_vehicle_simulation_gazebo.html

https://microsoft.github.io/AirSim/px4_multi_vehicle/

In [ ]:
# !cd PX4-Autopilot
# !Tools/sitl_multiple_run.sh -m iris -n 2 #-m: name of the model, -n: number of entities, -w: world

# !MicroXRCEAgent udp4 -p 8888
# # Once we run MicroXRCE we can see the topics of each drone instance referenced as /px4 1, /px4 2, etc...

From the **multivehicle simulation files** folder take the files and place (replace the already exisiting files if present) them in the following folders:
- *sitl_multiple_run.sh* --> ~/PX4-Autopilot/Tools/simulation/gazebo-classic
- *jinja_gen.py* --> ~/PX4-Autopilot/Tools/simulation/gazebo-classic/sitl_gazebo-classic
- *iris_depth_camera.sdf.jinja* --> ~/PX4-Autopilot/Tools/simulation/gazebo-classic/sitl_gazebo-classic/models/iris_depth_camera

From the **world** folder take the file and place it in ~/PX4-Autopilot/Tools/simulation/gazebo-classic/sitl_gazebo-classic/worlds. After that, go to ~PX4-Autopilot/src/modules/simulation/simulator_mavlink and open **sitl_targets_gazebo-classic.cmake** and write 'our_empty' among the names all other already existing worlds (from line 110 to 120)

In the *launch* directory in the *px4_offboard* package folder open the launch file. At line 15 replace *drone_ws* with the name of your ROS2 worksapce

**IF YOU WANT (NOT NECESSARY TO MAKE THE SIMULATION WORK)** 

Take the **meshes** folder from the *px4_offboard* package and place it in ~/NAME_OF_YOUR_WORKSPACE/install/px4_offboard/share/px4_offboard. This allows to see the drone's model in Rviz but the simulation works fine anyway.

**TO RUN THE SIMULATION** from terminal

In [ ]:
!cd ~/NAME_OF_YOUR_WORKSPACE
!colcon build #if you need to compile
!source ./install/setup.bash
!ros2 launch px4_offboard offboard_velocity_control.launch.py

The Gazebo simulation is launched by the python script *precesses.py* at ~/NAME_OF_YOUR_WORKSPACE/src/px4_offboard/px4_offboard with the command:

cd ~/PX4-Autopilot/Tools/simulation/gazebo-classic && ./sitl_multiple_run.sh -s iris_depth_camera:1:0:0,iris_depth_camera:1:3:3 -w our_empty

The syntax is --> ./sitl_multiple_run.sh -s MODEL1:N° OF ENTITES:X_COORD:Y_COORD,MODEL2:N° OF ENTITES:X_COORD:Y_COORD, ...

**BRIEF EXPLAINATION OF HOW ALL THIS WORKS**

In Gazebo/ROS when multiple entities exist, they must have different frames, topics ecc ecc names. For other models (like *iris*) everything is already managed by PX4 & Co (to each entity is assigned a namespace px4_1, px4_2 ecc,  so each topic name will start with /px4_N/...), but not for the *iris_depth_camera* model.

When the *sitl_multiple_run.sh* file is executed it calls the *jinja_gen.py* script that uses the *.sdf.jinja* file of the desired model to generate all the sdf files for all the entities: the sdf.jinja file is like a generic sdf file where the things that must be different among the entities to avoid conflicts are not assigned to a fixed value or have fixed names, but these values/names are received from the sitl_multiple_run.sh and jinja_gen.py files.

The sdf.jinja file for the iris_depth_camera model didn't exist, so i created one where I said that the base_link and camera_link frames and something more in the camera_plugin have names taken as parameters from the two scripts. In this way for example the first drone will have *base_link_1* and *camera_link_1* and the second one will have *base_link_2* and *camera_link_2* as frames names.

I also created two .urdf files with matching frames names to publish all the frames using the robot_state_publisher nodes.

With all this we are able to se the cameras data from all the drones and to create the topics refered to the map frame (pointcloud_wrt_map) to create the Octomap!!

**PROBLEMS**

In the single vehicle simulation the map->base_link transform was published starting form the data taken from the topic /fmu/out/vehicle_odometry but with 2 drones this doesn't work: the odometry corresponds to the relative position and orientation of the drone from its initial position and orientation, so if the drone doesn't move the odometry will be always 0 0 0 ... With two drones, both will have odometries equal to 0 0 0 if they don't move, so if we publish the transform map->base_link_1 and map->base_link_2 using the odometry the frames of the two drones will overlap. I tried using other topics like *local_position* but the result is the same. So we need the absolute position of the drones with respect to the world.

**POSSIBLE SOLUTIONS**

- NOT OPTIMAL BECAUSE COULD WORK ONLY IN SIMULATION, BUT **IN THEORY** EASY: in the *processes.py* instead of writing directly the coordinates of the drones, we can pass them as parameters from the *offboard_velocity_control.launch.py*, so that we can give them as parameters also to the *map_base_link_publisher* node and use them to adjust what we receive for the odometry topic
- BETTER BECAUSE IT SHOULD WORK ALSO WITH REAL DRONES, BUT MORE DIFFICULT: implement (or find somewhere) a localization algorithm that can compute the absolute position of each drone using sensors readings. Something already exists but i dont' know if it works with 3D sensor like depth camera and for drones in general
- I DON'T KNOW IF IT IS POSSIBLE: there is a topic called vechicle_global_position already provided by PX4 that gives latitude, longitude and altitude of the drone using the GPS and other things. I don't know if there is a way to convert those data into x,y and z coordinates of the drone